# RAG Integration

##### For the second agent, a deal price will be estimated using OpenAI, with extra guidance provided to improve accuracy.

# Libraries Import and Enviroment Init

In [ ]:
import os
import locale
import logging
from dotenv import load_dotenv
from huggingface_hub import login
import numpy as np
import re
from sentence_transformers import SentenceTransformer
import chromadb
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from litellm import completion
from tqdm.notebook import tqdm
from Agents_Folder.Evaluation_script import evaluate
from Agents_Folder.HuggingFace_Item_script import Item

In [2]:
load_dotenv(override=True)
DataBase = "products_vectorstore"

# Hugging Face Integration

In [3]:
hf_token = os.getenv("HF_TOKEN")
if hf_token:
    print("Logging in to Hugging Face Hub... Process is successful.")
    login(token=hf_token, add_to_git_credential=False)
else:
    print("Hugging Face API key not found. Please set the HF_TOKEN environment variable.")

Logging in to Hugging Face Hub... Process is successful.


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


# Dataset Settings, Mode Selection (Lite or Full) and ChromaDB

In [4]:
LITE_MODE = False
username="yasinemirkutlu44"
dataset = f"{username}/items_lite_eds" if LITE_MODE else f"{username}/items_full_eds"

train, val, test = Item.from_hub(dataset)

print(f"Loaded {len(train):,} training items, {len(val):,} validation items, {len(test):,} test items")

Loaded 800,000 training items, 10,000 validation items, 10,000 test items


In [5]:
# Creating ChromaDB to store embeddings vectors
chroma_client = chromadb.PersistentClient(path=DataBase)

# SentenceTransformer Encoder using Hugging Face

In [6]:
# all-MiniLM is a handy HuggingFace model that encodes sentences and paragraphs into 384-dimensional embeddings. It runs locally

embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Populate Chroma DB

In [7]:
# Check if the collection exists; if not, create it

collection_name = "products"
existing_collection_names = [collection.name for collection in chroma_client.list_collections()]

if collection_name not in existing_collection_names:
    collection = chroma_client.create_collection(collection_name)
    for i in tqdm(range(0, len(train), 1000)):
        documents = [item.summary for item in train[i: i+1000]]
        vectors = embedding_model.encode(documents).astype(float).tolist()
        metadatas = [{"category": item.category, "price": item.price} for item in train[i: i+1000]]
        ids = [f"doc_{j}" for j in range(i, i+1000)]
        ids = ids[:len(documents)]
        collection.add(ids=ids, documents=documents, embeddings=vectors, metadatas=metadatas)

collection = chroma_client.get_or_create_collection(collection_name)

# Visualise Vectorised Product Data

In [8]:
MAX_Data_Points_Visualised = 10_000

In [9]:
CATEGORIES = ['Appliances', 'Automotive', 'Cell_Phones_and_Accessories', 'Electronics','Musical_Instruments', 'Office_Products', 'Tools_and_Home_Improvement', 'Toys_and_Games']
COLOURS = ['cyan', 'blue', 'brown', 'orange', 'yellow', 'green' , 'purple', 'red']

In [10]:
# Specify which fields to return from ChromaDB
# Limit how many records to retrieve
result = collection.get(include=['embeddings', "documents", 'metadatas'],
    limit=MAX_Data_Points_Visualised) # Fetch up to MAXIMUM_DATAPOINTS items from the Chroma collection, including embeddings, documents, and metadata

# Convert the returned list of embedding vectors into a NumPy array (shape: [N, embedding_dim])
vectors = np.array(result['embeddings'])
# Extract the list of document texts (e.g., product summaries) returned from the collection
documents = result['documents']

# Pull the "category" field out of each metadata dictionary into a list
categories = [meta['category'] for meta in result['metadatas']]

# Map each category to a colour:
# 1) find the category's index in the CATEGORIES list
# 2) use that index to pick the corresponding entry in COLOURS

colours = [COLOURS[CATEGORIES.index(cat)] for cat in categories]


In [11]:
print("MAXIMUM_DATAPOINTS =", MAX_Data_Points_Visualised)

result = collection.get(include=["embeddings"], limit=MAX_Data_Points_Visualised)
print("keys:", result.keys())
print("num embeddings:", len(result["embeddings"]))
print("vectors shape:", np.array(result["embeddings"]).shape)


MAXIMUM_DATAPOINTS = 10000
keys: dict_keys(['ids', 'embeddings', 'documents', 'uris', 'included', 'data', 'metadatas'])
num embeddings: 10000
vectors shape: (10000, 384)


# 2D TSNE Chart

In [12]:
tsne = TSNE(n_components=2, random_state=42)
vectors_2d = tsne.fit_transform(vectors)

In [13]:
fig = go.Figure(data=[go.Scattergl(
    x=vectors_2d[:, 0],
    y=vectors_2d[:, 1],
    mode='markers',
    marker=dict(
        color=colours,
        size=3,
        opacity=0.7),
        text=[f"Category: {c}<br>Text: {d[:60]}..." for c, d in zip(categories, documents)],
        hoverinfo='text'
    )])

fig.update_layout(
    title="2D t-SNE Visualization of Product Embeddings",
    xaxis_title="t-SNE Dimension 1",
    yaxis_title="t-SNE Dimension 2",
    width=800,
    height=600,
    margin=dict(r=10, b=10, l=10, t=40)
)

# 3D TSNE Chart

In [14]:
tsne = TSNE(n_components=3, random_state=42)
vectors_2d = tsne.fit_transform(vectors)

In [15]:
fig = go.Figure(data=[go.Scatter3d(
    x=vectors_2d[:, 0],
    y=vectors_2d[:, 1],
    z =vectors_2d[:, 2],
    mode='markers',
    marker=dict(
        color=colours,
        size=2,
        opacity=0.7),
        text=[f"Category: {c}<br>Text: {d[:60]}..." for c, d in zip(categories, documents)],
        hoverinfo='text'
    )])

fig.update_layout(
    title="3D t-SNE Visualization of Product Embeddings",
    xaxis_title="t-SNE Dimension 1",
    yaxis_title="t-SNE Dimension 2",
    width=800,
    height=600,
    margin=dict(r=10, b=10, l=10, t=40)
)

In [16]:
test[0]

<Old Blood Noise Excess V2 Distortion Chorus/Delay Pedal = $219.0>

In [17]:
def vector(item):
    return embedding_model.encode(item.summary)


In [47]:
def find_similars(item):                       # define a function that finds items similar to `item`
    vec = vector(item)                         # convert `item.summary` into an embedding vector
    results = collection.query(                # search ChromaDB for nearest neighbors to that vector
        query_embeddings=vec.astype(float).tolist(),  # convert vector to a plain Python list of floats
        n_results=10                            # return the top 10 most similar items
    )
    documents = results["documents"][0][:]     # get the 10 matched document texts (first query’s results)
    prices = [m["price"] for m in results["metadatas"][0][:]]  # extract the price from each match’s metadata
    return documents, prices                   # return matched texts + their prices


In [48]:
find_similars(test[0])  # test the function on the first test item

(['Title: Old Blood Noise Endeavors Procession Reverb  \nCategory: Audio Effects  \nBrand: Old Blood Noise Endeavors  \nDescription: A compact, sci‑fi inspired reverb pedal with three modulation modes for creating otherworldly echo effects.  \nDetails: Features adjustable mix, decay, speed, depth, and footswitches for bypass and hold; powered by 9\u202fV DC with 60\u202fmA draw.',
  'Title: Boss MD‑2 Mega Distortion Modulation Multi‑Effects Pedal  \nCategory: Music Equipment → Effects Pedals  \nBrand: Boss  \nDescription: A powerful distortion pedal that delivers extreme low‑end crunch and endless sustain for metal and hard rock.  \nDetails: Features a Gain Boost circuit, bottom‑heavy Bottom control for 6/7‑string guitars, adjustable Tone knob, and 1\u202fMΩ input impedance.',
  'Title: Old Blood Noise Endeavors Mondegreen Delay Pedal  \nCategory: Musical Instruments / Effects Pedals  \nBrand: Old Blood Noise  \nDescription: A digital delay pedal that transforms your signal into creati

In [49]:
# This function is written to provide some context to OpenAI model by selecting 5 products with similar descriptions.
# The selected products and their prices will be included in the prompt to help the model make a more accurate price estimation.

def provide_context(similars, prices):
    message = "For context, here are some other items that might be similar to the item you need to estimate.\n\n"
    for similar, price in zip(similars, prices):
        message += f"Potentially related product:\n{similar}\nPrice is ${price:.2f}\n\n"
    return message

def provide_context(similars, prices):
    message = (
        "Use the other items that might be similar to the item you need to estimate. Each example includes a potentially similar product "
        "and its known price. You may want to consider the similar items to improve the prediction quality.\n\n"
    )
    for similar, price in zip(similars, prices):
        message += (
            f"Reference product description:\n{similar}\n"
            f"Known price: ${price:.2f}\n\n"
        )
    return message


In [35]:
documents, prices = find_similars(test[0])
print(provide_context(documents, prices))

Use the other items that might be similar to the item you need to estimate. Each example includes a potentially similar product and its known price. You may want to consider the similar items to improve the prediction quality.

Reference product description:
Title: Old Blood Noise Endeavors Procession Reverb  
Category: Audio Effects  
Brand: Old Blood Noise Endeavors  
Description: A compact, sci‑fi inspired reverb pedal with three modulation modes for creating otherworldly echo effects.  
Details: Features adjustable mix, decay, speed, depth, and footswitches for bypass and hold; powered by 9 V DC with 60 mA draw.
Known price: $209.00

Reference product description:
Title: Boss MD‑2 Mega Distortion Modulation Multi‑Effects Pedal  
Category: Music Equipment → Effects Pedals  
Brand: Boss  
Description: A powerful distortion pedal that delivers extreme low‑end crunch and endless sustain for metal and hard rock.  
Details: Features a Gain Boost circuit, bottom‑heavy Bottom control for 6

In [50]:
def message_for_model(item, similars, prices):
    message = (
        f"You are an expert price estimation model. Given the product description, "
        f"estimate the price of this product. Respond with the price only, no explanation.\n\n{item.summary}\n\n"
    )
    message += provide_context(similars, prices)
    return [{"role": "user", "content": message}]

In [51]:
documents, prices = find_similars(test[0])
print(message_for_model(test[0], documents, prices)[0]['content'])

You are an expert price estimation model. Given the product description, estimate the price of this product. Respond with the price only, no explanation.

Title: Excess V2 Distortion/Modulation Pedal  
Category: Music Pedals  
Brand: Old Blood Noise  
Description: A versatile pedal offering distortion and three modulation modes—delay, chorus, and harmonized fifths—with full control over signal routing and expression.  
Details: Features include separate gain, tone, and volume controls; time, depth, and volume per modulation; order switching, soft‑touch bypass, and expression jack for dynamic control.

Use the other items that might be similar to the item you need to estimate. Each example includes a potentially similar product and its known price. You may want to consider the similar items to improve the prediction quality.

Reference product description:
Title: Old Blood Noise Endeavors Procession Reverb  
Category: Audio Effects  
Brand: Old Blood Noise Endeavors  
Description: A com

In [52]:
def gpt_5_2_rag(item):
    documents, prices = find_similars(item)
    response = completion(model="gpt-5.2", messages=message_for_model(item, documents, prices), reasoning_effort="none", seed=42)
    return response.choices[0].message.content

In [53]:
test[0].price

219.0

In [54]:
gpt_5_2_rag(test[0])

'$249.00'

In [56]:
evaluate(gpt_5_2_rag, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$30 $24 $10 $24 $30 $70 $26 $81 $8 $45 $43 $50 $0 $4 $11 $0 $5 $20 $43 $30 $50 $9 $25 $15 $43 $193 $65 $2 $111 $52 $4 $20 $10 $15 $15 $70 $66 $26 $44 $22 $40 $5 $5 $25 $60 $2 $24 $1 $75 $8 $11 $95 $245 $5 $37 $4 $3 $49 $88 $7 $107 $38 $0 $50 $121 $10 $40 $265 $0 $19 $13 $8 $0 $6 $27 $13 $6 $3 $2 $1 $0 $9 $0 $39 $17 $0 $58 $14 $101 $11 $8 $25 $0 $5 $0 $31 $5 $59 $70 $109 $10 $13 $7 $26 $10 $8 $3 $281 $4 $120 $10 $15 $1 $29 $4 $0 $5 $5 $24 $247 $2 $40 $2 $2 $0 $36 $6 $21 $70 $29 $0 $23 $5 $0 $65 $1 $56 $15 $6 $5 $21 $99 $15 $5 $14 $43 $5 $80 $55 $5 $0 $16 $0 $10 $1 $1 $26 $4 $10 $0 $39 $16 $2 $1 $311 $2 $252 $20 $5 $1 $12 $5 $40 $12 $27 $21 $1 $30 $66 $12 $49 $15 $210 $28 $20 $17 $33 $12 $18 $1 $15 $4 $5 $41 $0 $0 $30 $12 $0 $9 

# Fine-tuned Llama 3.2 Model (Specialist Modal)

In [57]:
import modal
Pricer = modal.Cls.from_name("pricer-service", "Pricer")
pricer = Pricer()

In [58]:
def specialist_model(item):
    return pricer.price.remote(item.summary)

In [59]:
def get_product_price(reply):
    reply = reply.replace("$", "").replace(",", "")
    match = re.search(r"(\d+(\.\d{1,2})?)", reply)
    return float(match.group()) if match else None

In [60]:
specialist_model(test[0])

189.0

# 3rd Prediction Model (Gemini 3 Pro Preview)

In [ ]:
from openai import OpenAI

client = OpenAI(
  api_key=os.environ["GOOGLE_API_KEY"],
  base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)


In [82]:
import os
from litellm import completion

def gemini_flash_lite_rag(item):
    documents, prices = find_similars(item)
    response = completion(
        model="gemini/gemini-2.5-flash-lite",
        messages=message_for_model(item, documents, prices),
        seed=42,
    )
    return response.choices[0].message.content


# Ensemble Implementation

In [88]:
def ensemble_agent(item):
    price1 = get_product_price(gpt_5_2_rag(item))
    price2 = specialist_model(item)
    return price1 * 0.9 + price2 * 0.1

In [89]:
evaluate(ensemble_agent, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$24 $29 $12 $25 $28 $80 $13 $59 $9 $51 $11 $53 $1 $9 $8 $2 $3 $17 $39 $33 $38 $10 $25 $15 $66 $200 $116 $1 $96 $52 $3 $18 $19 $18 $13 $75 $62 $26 $50 $1 $54 $10 $4 $19 $65 $1 $22 $0 $71 $6 $11 $96 $249 $5 $50 $6 $8 $51 $77 $4 $109 $36 $1 $49 $68 $11 $41 $261 $1 $19 $14 $8 $9 $5 $27 $12 $15 $3 $1 $2 $6 $10 $3 $42 $15 $16 $56 $4 $85 $12 $8 $19 $0 $4 $0 $40 $0 $58 $71 $137 $4 $11 $7 $16 $6 $3 $4 $287 $3 $116 $12 $1 $3 $33 $6 $13 $5 $4 $22 $141 $1 $120 $0 $3 $2 $29 $4 $20 $56 $29 $12 $18 $7 $0 $74 $2 $65 $13 $13 $13 $18 $94 $15 $5 $10 $39 $6 $43 $54 $5 $0 $3 $2 $1 $5 $5 $30 $0 $19 $2 $34 $16 $1 $1 $334 $2 $127 $16 $3 $2 $11 $5 $56 $11 $28 $26 $1 $28 $56 $8 $80 $15 $197 $20 $17 $17 $37 $11 $20 $1 $16 $5 $2 $36 $4 $14 $25 $20 $3 $9 

In [90]:
## Neural network is not used because ensemble agent is performing well enough.

In [91]:
root = logging.getLogger()
root.setLevel(logging.INFO)

# Test on unseen data using GPT 5.2 Frontier Model

In [94]:
from agents.frontier_agent import FrontierAgent

agent = FrontierAgent(collection)
agent.price("Quadcast HyperX condenser mic, connects via usb-c to your computer for crystal clear audio")

INFO:root:[OpenAI GPT 5.2 Frontier Agent] Initializing Frontier Agent
INFO:root:[OpenAI GPT 5.2 Frontier Agent] OpenAI GPT 5.2 Frontier Agent is setting up with OpenAI
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:root:[OpenAI GPT 5.2 Frontier Agent] Frontier Agent is ready
INFO:root:[OpenAI GPT 5.2 Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[OpenAI GPT 5.2 Frontier Agent] Frontier Agent has found similar products
INFO:root:[OpenAI GPT 5.2 Frontier Agent] Frontier Agent is about to call gpt-5.2 with context including 5 similar products
INFO:root:[OpenAI GPT 5.2 Frontier Agent] Frontier Agent completed - predicting $129.99


129.99

In [95]:
agent.price("Apple MacBook Pro 16-inch with M2 Max chip, 32GB RAM, 1TB SSD, Silver")

INFO:root:[OpenAI GPT 5.2 Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[OpenAI GPT 5.2 Frontier Agent] Frontier Agent has found similar products
INFO:root:[OpenAI GPT 5.2 Frontier Agent] Frontier Agent is about to call gpt-5.2 with context including 5 similar products
INFO:root:[OpenAI GPT 5.2 Frontier Agent] Frontier Agent completed - predicting $2999.00


2999.0

# Test on unseen data using Ensemble

In [96]:
from agents.ensemble_agent import EnsembleAgent
agent = EnsembleAgent(collection)

INFO:root:[Ensemble Agent] Initializing Ensemble Agent
INFO:root:[Specialist Agent] Specialist Agent is initializing - connecting to modal
INFO:root:[OpenAI GPT 5.2 Frontier Agent] Initializing Frontier Agent
INFO:root:[OpenAI GPT 5.2 Frontier Agent] OpenAI GPT 5.2 Frontier Agent is setting up with OpenAI
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:root:[OpenAI GPT 5.2 Frontier Agent] Frontier Agent is ready
INFO:root:[Ensemble Agent] Ensemble Agent is ready


In [97]:
agent.price("Apple MacBook Pro 16-inch with M2 Max chip, 32GB RAM, 1TB SSD, Silver")

INFO:root:[Ensemble Agent] Running Ensemble Agent - preprocessing text
00:47:03 - LiteLLM:INFO: utils.py:3421 - 
LiteLLM completion() model= openai/gpt-oss-20b; provider = groq
INFO:LiteLLM:
LiteLLM completion() model= openai/gpt-oss-20b; provider = groq
00:47:03 - LiteLLM:INFO: utils.py:1302 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:root:[Ensemble Agent] Pre-processed text using groq/openai/gpt-oss-20b
INFO:root:[Specialist Agent] Specialist Agent is calling remote fine-tuned model
INFO:root:[Specialist Agent] Specialist Agent completed - predicting $220.00
INFO:root:[OpenAI GPT 5.2 Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[OpenAI GPT 5.2 Frontier Agent] Frontier Agent has found similar products
INFO:root:[OpenAI GPT 5.2 Frontier Agent] Frontier Agent is about to call gpt-5.2 with context including 5 similar products
INFO:root:[OpenAI GPT 5.2 Frontier Agent] Frontier Agent completed - predicting $3499.00
INFO:root:[Ensemble Agent] Ensemble Agent complete - returning $3171.10


3171.1